# 25.07.21 오후 수업   premium.csv 데이터를 활용한 선형 회귀

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 1. 데이터 로드 및 준비
# CSV 파일을 pandas DataFrame으로 읽어옵니다.
try:
    df = pd.read_csv('C:/githome/9-11week_py_statistics/data1/premium.csv')
except UnicodeDecodeError:
    # 인코딩 에러 발생 시 다른 인코딩으로 시도
    df = pd.read_csv('C:/githome/9-11week_py_statistics/data1/premium.csv', encoding='cp949')

# 결측치 처리 (bmi 컬럼의 누락된 값을 평균으로 채움)
df['bmi'].fillna(df['bmi'].mean(), inplace=True)

# 특성(X)과 타겟(y) 분리
X = df.drop('charges', axis=1)
y = df['charges']

# 범주형 데이터와 수치형 데이터 컬럼 정의
categorical_features = ['sex', 'smoker', 'region']
numerical_features = ['age', 'bmi', 'children']

# 전처리 파이프라인 설정
# - OneHotEncoder: 범주형 데이터를 수치형으로 변환
# - passthrough: 수치형 데이터는 그대로 사용
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# 2. 교차 검증을 사용하지 않은 모델
print("--- 교차 검증 미적용 모델 ---")
# 데이터를 훈련 세트와 테스트 세트로 8:2 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 전처리 및 모델 학습을 위한 파이프라인 생성
pipeline_simple = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('regressor', LinearRegression())])

# 모델 학습
pipeline_simple.fit(X_train, y_train)

# 예측 및 평가
y_pred = pipeline_simple.predict(X_test)
mse_simple = mean_squared_error(y_test, y_pred)
r2_simple = r2_score(y_test, y_pred)

print(f"테스트 세트에서의 평균 제곱 오차 (MSE): {mse_simple:.2f}")
print(f"테스트 세트에서의 결정 계수 (R²): {r2_simple:.4f}")
print("-" * 30)

# 3. 교차 검증을 사용한 모델
print("\n--- 교차 검증 적용 모델 ---")
# 전처리 및 모델을 포함하는 전체 파이프라인
pipeline_cv = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', LinearRegression())])

# 5-겹 교차 검증 수행
# MSE (음수로 반환되므로 -1 곱함)
cv_mse_scores = -cross_val_score(pipeline_cv, X, y, cv=5, scoring='neg_mean_squared_error')
# R²
cv_r2_scores = cross_val_score(pipeline_cv, X, y, cv=5, scoring='r2')

print(f"교차 검증 각 폴드의 MSE 점수: \n{cv_mse_scores}")
print(f"교차 검증 평균 MSE: {cv_mse_scores.mean():.2f} (표준편차: {cv_mse_scores.std():.2f})")
print("\n" + "-"*15 + "\n")
print(f"교차 검증 각 폴드의 R² 점수: \n{cv_r2_scores}")
print(f"교차 검증 평균 R²: {cv_r2_scores.mean():.4f} (표준편차: {cv_r2_scores.std():.4f})")

--- 교차 검증 미적용 모델 ---
테스트 세트에서의 평균 제곱 오차 (MSE): 33627022.07
테스트 세트에서의 결정 계수 (R²): 0.7834
------------------------------

--- 교차 검증 적용 모델 ---
교차 검증 각 폴드의 MSE 점수: 
[37333420.30374555 38252383.45633688 32962417.93798563 39689926.68963022
 37118909.96424668]
교차 검증 평균 MSE: 37071411.67 (표준편차: 2245263.88)

---------------

교차 검증 각 폴드의 R² 점수: 
[0.75975542 0.70548863 0.77540897 0.73263652 0.75561818]
교차 검증 평균 R²: 0.7458 (표준편차: 0.0244)


C:\Users\Admin\AppData\Local\Temp\ipykernel_130556\307068176.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bmi'].fillna(df['bmi'].mean(), inplace=True)
